In [1]:
import tensorflow
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input,decode_predictions
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc

In [2]:
def vgg_model():
    model = VGG16(weights='imagenet')
    int_model = tensorflow.keras.Model(model.inputs, [model.layers[5].output, model.layers[17].output])
    return int_model


def resnet_model():
    model = ResNet50(weights='imagenet')
    int_model = tensorflow.keras.Model(model.inputs, [model.layers[14].output, model.layers[168].output])
    return int_model


def inception_model():
    model = InceptionV3(weights='imagenet')
    int_model = tensorflow.keras.Model(model.inputs, [model.layers[31].output, model.layers[284].output])
    return int_model


def features_for_img(img_path, int_model, size):
    img = plt.imread(img_path)[:, :, :3]
    img = tensorflow.image.resize(img, [size, size]).numpy()
    img = np.expand_dims(img, 0)
    
    low, high = int_model.predict(img)
    return low, high


def get_features_for_task(base_path, int_model, size):
    train_base_path = base_path + 'train/'
    base_path = base_path + 'test/'
    
    train_wugs_low = []
    train_wugs_high = []
    for i in range(1, 6):
        curr_path = train_base_path + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model, size)
        train_wugs_low.append(low)
        train_wugs_high.append(high)
        
    wugs_low = []
    wugs_high = []
    for i in range(1, 6):
        curr_path = base_path + 'in_' + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model, size)
        wugs_low.append(low)
        wugs_high.append(high)

    not_wugs_low_close = []
    not_wugs_high_close = []
    for i in range(1, 6):
        curr_path = base_path + 'out_close_' + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model, size)
        not_wugs_low_close.append(low)
        not_wugs_high_close.append(high)
        
    not_wugs_low_far = []
    not_wugs_high_far = []
    for i in range(1, 6):
        curr_path = base_path + 'out_far_' + str(i) + '_fin.png'
        low, high = features_for_img(curr_path, int_model, size)
        not_wugs_low_far.append(low)
        not_wugs_high_far.append(high)
        
    return (train_wugs_low, train_wugs_high, wugs_low, wugs_high, not_wugs_low_close, not_wugs_high_close, not_wugs_low_far, not_wugs_high_far)


In [5]:
int_model = vgg_model()

In [6]:
# Please replace with new path.
features = get_features_for_task('/viscam/u/joycj/geoclidean/dataset/geoclidean/constraints/concept_cct/', int_model, 224)
